# Fire response traits in plants from field samples 
R code to calculate plant traits from field data

## Query the database

In [36]:
library(RPostgreSQL)
library(ggplot2)
##library(forcats)
library(dplyr)
#library(data.table)
require(tidyr)


Loading required package: tidyr



In [3]:
if (file.exists("../secrets/database.ini")) {
  tmp <- readLines("../secrets/database.ini")
  tmp <- data.frame(strsplit(tmp,"="))
  dbinfo <- tmp[2,]
  names(dbinfo) <- tmp[1,]
   rm(tmp)
} else {
  cat("No database information found")
}

In [4]:
drv <- dbDriver("PostgreSQL") ## remember to update .pgpass file
con <- dbConnect(drv, dbname = dbinfo$database,
                 host = dbinfo$host, port = dbinfo$port,
                 user = dbinfo$user)

In [5]:
qry <- "SELECT * FROM form.quadrat_samples"
my.table<- dbGetQuery(con, qry)
str(my.table)
#Close PostgreSQL connection
dbDisconnect(con)


'data.frame':	13145 obs. of  20 variables:
 $ record_id             : int  842 843 844 845 846 847 848 849 850 851 ...
 $ visit_id              : chr  "AlpAsh_68" "AlpAsh_68" "AlpAsh_68" "AlpAsh_68" ...
 $ visit_date            : Date, format: "2021-04-12" "2021-04-12" ...
 $ sample_nr             : int  1 1 1 1 1 1 1 1 1 1 ...
 $ species               : chr  "Eucalyptus dalrympleana subsp. dalrympleana" "Eucalyptus delegatensis subsp. delegatensis" "Eucalyptus delegatensis subsp. delegatensis" "Acacia obliquinervia" ...
 $ species_code          : int  7309 6885 6885 3837 14811 2827 8642 1591 9968 5444 ...
 $ species_notes         : chr  NA NA NA NA ...
 $ resprout_organ        : chr  "Lignotuber" "None" "None" "Basal" ...
 $ seedbank              : chr  "Canopy" "Canopy" "Canopy" NA ...
 $ adults_unburnt        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ resprouts_live        : num  3 0 0 137 175 10 55 7 0 1 ...
 $ resprouts_died        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ resprouts_kill        : 

[1] TRUE

## Resprout organ

In [50]:
my.table %>% 
    group_by(species) %>% 
    summarise(total=n()) -> totals

my.table %>% 
    group_by(species,resprout_organ) %>% 
    summarise(total=n()) %>% 
    pivot_wider(names_from=resprout_organ,values_from=total, values_fill = 0) %>% 
    left_join(totals,by="species") %>% ungroup -> resprout_organ


`summarise()` has grouped output by 'species'. You can override using the `.groups` argument.



### Seeders

In [58]:
resprout_organ %>% filter(None>0 & None+`NA` == total) 

species,NA,None,Basal,Stolon,Tuber,Short rhizome,Epicormic,Tussock,Apical,Lignotuber,total
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Abutilon oxycarpum,0,1,0,0,0,0,0,0,0,0,1
Acacia elata,4,2,0,0,0,0,0,0,0,0,6
Acacia irrorata,2,32,0,0,0,0,0,0,0,0,34
Acacia mearnsii,0,3,0,0,0,0,0,0,0,0,3
Acacia obtusifolia,7,1,0,0,0,0,0,0,0,0,8
Acacia suaveolens,3,2,0,0,0,0,0,0,0,0,5
Acacia verniciflua,0,4,0,0,0,0,0,0,0,0,4
Ageratina riparia,0,1,0,0,0,0,0,0,0,0,1
Almaleea incurvata,0,7,0,0,0,0,0,0,0,0,7


## Seedbank

In [11]:
sb <-table(my.table$species, my.table$seedbank)
head(sb)


                    
                     Canopy Non-canopy Soil-persistent Transient
  ?Lastreopsis spp.       0          0               0         2
  Abutilon oxycarpum      0          0               0         0
  Acacia cognata          0          0               4         0
  Acacia disparrima       0          0               0         0
  Acacia elata            0          0               6         0
  Acacia falciformis      0          0               8         0

In [14]:
#N total live resprouts
plant.traits<- with(my.table,
          aggregate(data.frame(N1=resprouts_live,
                               N2=resprouts_reproductive,
                               N5=recruits_live,
                               N6=recruits_reproductive,
                               N7=resprouts_died,
                               N8=recruits_died,
                               N9=resprouts_kill),
                               by=list(species=species), sum, na.rm=T))

In [15]:
head(plant.traits)

,species,N1,N2,N5,N6,N7,N8,N9
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,?Lastreopsis spp.,8,0,0,0,0,0,0
2,Abutilon oxycarpum,0,0,5,0,0,0,0
3,Acacia cognata,0,0,24,0,0,0,0
4,Acacia disparrima,1,0,54,0,0,0,0
5,Acacia elata,1,0,16,0,0,0,4
6,Acacia falciformis,88,0,39,0,0,0,30


In [20]:
plant.traits %>% mutate(n_fire_mortality= (N1+N7+N9),
                        fire_mortality=(100*N9)/n_fire_mortality) %>% filter(n_fire_mortality>0) %>% slice_head(n=4)


species,N1,N2,N5,N6,N7,N8,N9,n_fire_mortality,fire_mortality
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
?Lastreopsis spp.,8,0,0,0,0,0,0,8,0.00000
Acacia disparrima,1,0,54,0,0,0,0,1,0.00000
Acacia elata,1,0,16,0,0,0,4,5,80.00000
Acacia falciformis,88,0,39,0,0,0,30,118,25.42373


In [21]:
plant.traits %>% mutate(n= (N1+N7),
                        sprout_surv=(100*N1)/n) %>% filter(n>0) %>% slice_head(n=4)


species,N1,N2,N5,N6,N7,N8,N9,n,sprout_surv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
?Lastreopsis spp.,8,0,0,0,0,0,0,8,100
Acacia disparrima,1,0,54,0,0,0,0,1,100
Acacia elata,1,0,16,0,0,0,4,1,100
Acacia falciformis,88,0,39,0,0,0,30,88,100


In [23]:
plant.traits %>% transmute(species,
                           seed_adult= (N5+N8)/(N1+N7),
                        n_recruit_surv=N5+N8,
                        recruit_surv=(100*N5)/n_recruit_surv,
                        n_reprod_resprout=N1,
                        n_reprod_recruits=N6,# the formula said N5 but this is the number of lives recruits
                        reprod_recruits=100*(N6/N5)) %>% slice_head(n=4)

                        # surv_dens=N2/area?
                        # recruit_dens=N5/area?


species,seed_adult,n_recruit_surv,recruit_surv,n_reprod_resprout,n_reprod_recruits,reprod_recruits
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
?Lastreopsis spp.,0,0,NaN,8,0,NaN
Abutilon oxycarpum,Inf,5,100,0,0,0
Acacia cognata,Inf,24,100,0,0,0
Acacia disparrima,54,54,100,1,0,0


# Seeders

1. Search spp. without reprout organ (seeder)
2. create a subseb for seeder
3. See number of recruit reproductive
4. Calculate time lapse beetween last fire and sampling date

In [31]:
as.data.frame(ro) #%>% filter(None>0)

Var1,Var2,Freq
<fct>,<fct>,<int>
?Lastreopsis spp.,Apical,0
Abutilon oxycarpum,Apical,0
Acacia cognata,Apical,0
Acacia disparrima,Apical,0
Acacia elata,Apical,0
Acacia falciformis,Apical,0
Acacia implexa,Apical,0
Acacia irrorata,Apical,0
Acacia maidenii,Apical,0


In [ ]:

#Crate a new data frame subsetting plant without reprout organ
seeders <- subset(plant.traits, none == 1)
str(seeders)
data.frame(seeders$plant_species1, seeders$none, seeders$rhizoma)

#Some plots
#Number of reproductive recruits
seeders %>%
  mutate(name = fct_reorder(plant_species1, n.reprod.recruit)) %>%
  ggplot(aes(x=plant_species1, y=n.reprod.recruit)) +
  geom_bar(stat="identity", fill="#f68060", alpha=.6, width=.4) +
  coord_flip() +
  xlab("") +
  theme_bw()
#There is none reproductive recruits is too soon I guess because there are
#live recruits
#Live recruit density
seeders %>%
  mutate(name = fct_reorder(plant_species1, recruit.dens)) %>%
  ggplot(aes(x=plant_species1, y= recruit.dens)) +
  geom_bar(stat="identity", fill="#f68060", alpha=.6, width=.4) +
  coord_flip() +
  xlab("") +
  theme_bw()

#Type of seedbank
str(my.table)

#Seed bank for seeders
myplot<-ggplot(my.table, aes(species, seedbank, fill=seedbank)) +
  geom_bar(stat="identity", position=position_dodge()) +
  coord_flip()

myplot %+% subset(my.table, resprout_organ %in% c("none"))

seeders %>%
  mutate(name = fct_reorder(plant_species1, soil_persistent)) %>%
  ggplot(aes(x=plant_species1, y=  soil_persistent)) +
  geom_bar(stat="identity", fill="#f68060", alpha=.6, width=.4) +
  coord_flip() +
  xlab("") +
  theme_bw()



# RESPROUTERS

- For spp. with resprout organ = rhizome
- Calculate % reprout alive (resprout alive/ live + died + kill)


In [ ]:

table(plant.traits$rhizoma)
my.rhizoma <- subset(plant.traits, rhizoma ==1)
str(my.rhizoma)
#14 obs. of  34 variables
table(my.rhizoma$plant_species1, my.rhizoma$rhizoma)
str(my.rhizoma)
write.csv(my.rhizoma, "my.rhizoma.csv")

#Some plots
my.rhizoma %>%
  mutate(name = fct_reorder(plant_species1, surv.dens)) %>%
  ggplot(aes(x=plant_species1, y=surv.dens)) +
  geom_bar(stat="identity", fill="#f68060", alpha=.6, width=.4) +
  coord_flip() +
  xlab("") +
  theme_bw()

rhizoma.traits %>%
  mutate(name = fct_reorder(plant_species, n.reprod.resprout)) %>%
  ggplot(aes(x=plant_species, y= n.reprod.resprout)) +
  geom_bar(stat="identity", fill="#f68060", alpha=.6, width=.4) +
  coord_flip() +
  xlab("") +
  theme_bw()
